In [14]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from reservoirpy.nodes import Reservoir
from sklearn.model_selection import train_test_split

In [11]:
df_bdu = pd.read_csv('bdu_clean.csv')
df_bdu.drop(['month_name', 'high_wind_event'])

,tmpf,dwpf,relh,drct,sknt,p01i,alti,vsby,skyc1,feel,date,gust_factor,high_wind_event,month_name,year,month,ext_gust_12h
0,53.6,50.0,87.56,0.0,0.0,0.0,30.14,10.0,SCT,53.60,2010-06-14 11:01:00,7.279661,0,June,2010,6,0
1,55.4,50.0,81.99,60.0,3.0,0.0,30.13,10.0,SCT,55.40,2010-06-14 11:20:00,10.531427,0,June,2010,6,0
2,59.0,51.8,76.98,60.0,4.0,0.0,30.13,10.0,SCT,59.00,2010-06-14 11:41:00,11.615349,0,June,2010,6,0
3,59.0,51.8,76.98,80.0,4.0,0.0,30.13,10.0,BKN,59.00,2010-06-14 12:00:00,11.615349,0,June,2010,6,0
4,60.8,51.8,72.20,0.0,0.0,0.0,30.11,10.0,SCT,60.80,2010-06-14 12:21:00,7.279661,0,June,2010,6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
377726,44.6,21.2,39.06,40.0,5.0,0.0,29.67,10.0,CLR,41.36,2025-01-10 22:35:00,12.699271,0,January,2025,1,0
377727,48.2,23.0,36.78,30.0,3.0,0.0,29.66,10.0,CLR,47.21,2025-01-10 22:55:00,10.531427,0,January,2025,1,0
377728,46.4,23.0,39.36,0.0,0.0,0.0,29.65,10.0,CLR,46.40,2025-01-10 23:15:00,7.279661,0,January,2025,1,0
377729,46.4,23.0,39.36,90.0,3.0,0.0,29.63,10.0,CLR,45.15,2025-01-10 23:35:00,10.531427,0,January,2025,1,0


In [16]:
def preprocess_time_features(X):
    if isinstance(X, pd.DataFrame):  # If data is a DataFrame
        X = X.copy()  # Avoid modifying original data
        for col in X.select_dtypes(include=['datetime64', 'object']):  # Convert datetime columns
            X[col] = pd.to_datetime(X[col]).astype(int) / 10**9  # Convert to seconds
    return X

In [17]:
df_bdu = preprocess_time_features(df_bdu)

/var/folders/l5/7h6z_dtn5yq32_cn1k36l3h00000gn/T/ipykernel_22005/2842116557.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  X[col] = pd.to_datetime(X[col]).astype(int) / 10**9  # Convert to seconds


DateParseError: Unknown datetime string format, unable to parse: SCT, at position 0

In [ ]:
dx_x = df_bdu['tmpf', 'dwpf', 'relh', 'drct', 'sknt', 'p01i', 'alti', 'vsby', 'skyc1',
       'feel', 'date', 'gust_factor', 'high_wind_event', 'month_name', 'year',
       'month', 'ext_gust_12h']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_x, df_y, test_size=0.15, random_state=42)

In [13]:
scaler_X = StandardScaler()

X_train = scaler_X.fit_transform(X_train)
X_test = scaler_X.transform(X_test)

y_train = preprocess_labels(y_train)
y_test = preprocess_labels(y_test)

reservoir = Reservoir(units=500, sr=0.8)  
X_train_reservoir = reservoir.run(X_train)  

classifier = LogisticRegression()
classifier.fit(X_train_reservoir, y_train)

X_test_reservoir = reservoir.run(X_test)
y_pred_probs = classifier.predict_proba(X_test_reservoir)[:, 1] 
y_pred = (y_pred_probs > 0.5).astype(int) 

# 🔹 Evaluate Performance
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"🔹 Accuracy: {accuracy:.3f}")
print(f"🔹 Precision: {precision:.3f}")
print(f"🔹 Recall: {recall:.3f}")
print(f"🔹 F1 Score: {f1:.3f}")

NameError: name 'X_train' is not defined